In [1]:
# dependencies
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np

In [2]:
# load data
batter = pd.read_csv('data/xander_bogaerts.csv')

# check columns
print(batter.columns)

# preview data
batter.head()

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2020-09-27,86.2,-2.42,3.73,Xander Bogaerts,593428,503285,home_run,hit_into_play_score,...,1,2,2,1,2,1,2,1,Standard,Standard
1,SI,2020-09-27,86.8,-2.15,3.59,Xander Bogaerts,593428,503285,NaN,ball,...,1,2,2,1,2,1,2,1,Standard,Standard
2,SI,2020-09-27,95.5,-2.88,5.37,Xander Bogaerts,593428,669060,single,hit_into_play_no_out,...,1,1,1,1,1,1,1,1,Standard,Standard
3,SI,2020-09-27,93.3,-2.86,5.38,Xander Bogaerts,593428,669060,NaN,foul,...,1,1,1,1,1,1,1,1,Standard,Standard
4,SL,2020-09-27,89.4,-2.86,5.30,Xander Bogaerts,593428,669060,NaN,foul,...,1,1,1,1,1,1,1,1,Standard,Standard


In [3]:
# list of necessary columns
cols_needed = ['p_throws', 'pitch_type', 'release_pos_x', 'release_pos_z', 'release_speed', 
               'effective_speed', 'release_spin_rate', 'release_extension',
               'plate_x', 'plate_z', 'balls', 'strikes', 'woba_denom', 'events', 'description']

# subset columns
batter_data = batter[cols_needed]

# preview data
batter_data.head()

,p_throws,pitch_type,release_pos_x,release_pos_z,release_speed,effective_speed,release_spin_rate,release_extension,plate_x,plate_z,balls,strikes,woba_denom,events,description
0,R,FF,-2.42,3.73,86.2,87.0,2439.0,6.8,0.06,3.04,1,0,1.0,home_run,hit_into_play_score
1,R,SI,-2.15,3.59,86.8,87.2,2209.0,6.8,1.51,2.48,0,0,NaN,NaN,ball
2,R,SI,-2.88,5.37,95.5,96.0,2035.0,6.5,-0.85,2.00,2,2,1.0,single,hit_into_play_no_out
3,R,SI,-2.86,5.38,93.3,93.6,2081.0,6.5,-0.81,1.68,2,2,NaN,NaN,foul
4,R,SL,-2.86,5.30,89.4,90.2,2257.0,6.6,-0.14,3.18,2,2,NaN,NaN,foul


In [4]:
# process to replace description types with base types

# list different description values
print('Original Counts')
print(batter_data['description'].value_counts())
print()

# build list of strikes types
strikes = ['called_strike', 'foul', 'swinging_strike', 'foul_tip', 'foul_bunt', 'swinging_strike_blocked']

# build list of balls types
balls = ['ball', 'blocked_ball']

# build list of in play types
in_play = ['hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score']

# conditional replace
batter_data.loc[batter_data['description'].isin(strikes), 'description'] = 'strike'
batter_data.loc[batter_data['description'].isin(balls), 'description'] = 'ball'
batter_data.loc[batter_data['description'].isin(in_play), 'description'] = 'in_play'

# check dataframe
print('Updated Counts')
print(batter_data['description'].value_counts())

Original Counts
ball                       1305
called_strike               824
foul                        594
hit_into_play               392
swinging_strike             300
hit_into_play_no_out        160
hit_into_play_score         109
blocked_ball                100
foul_tip                     27
swinging_strike_blocked      16
hit_by_pitch                  2
Name: description, dtype: int64

Updated Counts
strike          1761
ball            1405
in_play          661
hit_by_pitch       2
Name: description, dtype: int64


In [5]:
# list different events values
print('Original Counts')
print(batter_data['events'].value_counts())
print()

# convert different out types to base type
outs = ['field_out', 'force_out', 'grounded_into_double_play', 'double_play', 'field_error', 'sac_fly', 'sac_bunt', 'fielders_choice', 'fielders_choice_out']

# conditional replace
batter_data.loc[batter_data['events'].isin(outs), 'events'] = 'out'

# convert caught stealing to like values
cs = ['caught_stealing_2b', 'caught_stealing_3b', 'caught_stealing_home']

# conditional replace
batter_data.loc[batter_data['events'].isin(cs), 'events'] = 'cs'

# replace strikeout/walk events with ball/strike
batter_data['events'] = np.where((batter_data['woba_denom'] == 1) & (batter_data['description'] == 'strike'), 'strike', batter_data['events'])
batter_data['events'] = np.where((batter_data['woba_denom'] == 1) & (batter_data['description'] == 'ball'), 'ball', batter_data['events'])

print('Updated Counts')
print(batter_data['events'].value_counts())
print()

Original Counts
field_out                    359
strikeout                    162
single                       147
walk                          93
double                        60
home_run                      44
force_out                     19
grounded_into_double_play     14
field_error                    7
sac_fly                        7
fielders_choice                3
hit_by_pitch                   2
caught_stealing_2b             2
fielders_choice_out            1
caught_stealing_3b             1
strikeout_double_play          1
Name: events, dtype: int64

Updated Counts
out             410
strike          163
single          147
ball             93
double           60
home_run         44
cs                3
hit_by_pitch      2
Name: events, dtype: int64



In [6]:
# if events is null, change to description procedure

# conditional replace
batter_data['events'] = np.where(batter_data['events'].isna(), batter_data['description'], batter_data['events'])

# check values
print('Updated Counts')
print(batter_data['events'].value_counts())
print()

# check data
batter_data.head()

Updated Counts
strike          1760
ball            1403
out              410
single           147
double            60
home_run          44
cs                 3
hit_by_pitch       2
Name: events, dtype: int64



,p_throws,pitch_type,release_pos_x,release_pos_z,release_speed,effective_speed,release_spin_rate,release_extension,plate_x,plate_z,balls,strikes,woba_denom,events,description
0,R,FF,-2.42,3.73,86.2,87.0,2439.0,6.8,0.06,3.04,1,0,1.0,home_run,in_play
1,R,SI,-2.15,3.59,86.8,87.2,2209.0,6.8,1.51,2.48,0,0,NaN,ball,ball
2,R,SI,-2.88,5.37,95.5,96.0,2035.0,6.5,-0.85,2.00,2,2,1.0,single,in_play
3,R,SI,-2.86,5.38,93.3,93.6,2081.0,6.5,-0.81,1.68,2,2,NaN,strike,strike
4,R,SL,-2.86,5.30,89.4,90.2,2257.0,6.6,-0.14,3.18,2,2,NaN,strike,strike


In [7]:
# drop extra columns and cs rows
batter_data.drop(columns = ['woba_denom', 'description'], inplace = True, axis = 1)

batter_dropcs = batter_data[batter_data['events'] != 'cs']

# export data
batter_dropcs.to_csv('data/batter_clean.csv', index = False)

In [8]:
# repeat process for pitcher data

# load data
pitcher = pd.read_csv('data/zac_gallen.csv')

# check columns
print(pitcher.columns)
print()

# preview data
pitcher.head()

# list of necessary columns
cols_needed = ['p_throws', 'pitch_type', 'release_pos_x', 'release_pos_z', 'release_speed', 
               'effective_speed', 'release_spin_rate', 'release_extension',
               'plate_x', 'plate_z', 'balls', 'strikes', 'woba_denom', 'events', 'description']

# subset columns
pitcher_data = pitcher[cols_needed]

# preview data
pitcher_data.head()

# process to replace description types with base types

# list different description values
print('Original Counts')
print(pitcher_data['description'].value_counts())
print()

# build list of strikes types
strikes = ['called_strike', 'foul', 'swinging_strike', 'foul_tip', 'foul_bunt', 'swinging_strike_blocked']

# build list of balls types
balls = ['ball', 'blocked_ball']

# build list of in play types
in_play = ['hit_into_play', 'hit_into_play_no_out', 'hit_into_play_score']

# conditional replace
pitcher_data.loc[pitcher_data['description'].isin(strikes), 'description'] = 'strike'
pitcher_data.loc[pitcher_data['description'].isin(balls), 'description'] = 'ball'
pitcher_data.loc[pitcher_data['description'].isin(in_play), 'description'] = 'in_play'

# check dataframe
print('Updated Counts')
print(pitcher_data['description'].value_counts())
print()

# list different events values
print('Original Counts')
print(pitcher_data['events'].value_counts())
print()

# convert different out types to base type
outs = ['field_out', 'force_out', 'grounded_into_double_play', 'double_play', 'field_error', 'sac_fly', 'sac_bunt', 'fielders_choice', 'fielders_choice_out']

# conditional replace
pitcher_data.loc[pitcher_data['events'].isin(outs), 'events'] = 'out'

# convert caught stealing to like values
cs = ['caught_stealing_2b', 'caught_stealing_3b', 'caught_stealing_home']

# conditional replace
pitcher_data.loc[pitcher_data['events'].isin(cs), 'events'] = 'cs'

# replace strikeout/walk events with ball/strike
pitcher_data['events'] = np.where((pitcher_data['woba_denom'] == 1) & (pitcher_data['description'] == 'strike'), 'strike', pitcher_data['events'])
pitcher_data['events'] = np.where((pitcher_data['woba_denom'] == 1) & (pitcher_data['description'] == 'ball'), 'ball', pitcher_data['events'])

print('Updated Counts')
print(pitcher_data['events'].value_counts())
print()

# if events is null, change to description procedure

# conditional replace
pitcher_data['events'] = np.where(pitcher_data['events'].isna(), pitcher_data['description'], pitcher_data['events'])

# check values
print('Updated Counts')
print(pitcher_data['events'].value_counts())
print()

# check data
pitcher_data.head()

# drop extra columns and cs rows
pitcher_data.drop(columns = ['woba_denom', 'description'], inplace = True, axis = 1)

pitcher_dropcs = pitcher_data[pitcher_data['events'] != 'cs']

# export data
pitcher_dropcs.to_csv('data/pitcher_clean.csv', index = False)

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima